In [1]:
import numpy as np

In [2]:
def load_usps(fn):
    with open(fn,"r") as f:
        f.readline()
        data = [[float(x) for x in l.split()] for l in f if len(l.split())>2]
    tmp=np.array(data)
    return tmp[:,1:],tmp[:,0].astype(int)

In [14]:
def ajout_bruit(X):
    """
    ajout du bruit w0 à la premiere colonne de 
    chaque exemple de la base X
    """
    return np.hstack((np.ones((len(X), 1)), X))

In [15]:
Xtrain,Ytrain = load_usps("USPS_train.txt")
Xtest,Ytest = load_usps("USPS_test.txt")

Xtrain = ajout_bruit(Xtrain)
Xtest = ajout_bruit(Xtest)

In [80]:
Xtrain[0]

array([1.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.369,
       1.862, 0.833, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.008, 1.297, 2.   , 1.307,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.59 , 2.   , 1.986, 0.435, 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.317, 1.825, 2.   , 1.562, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.062, 1.54 , 2.   ,
       1.778, 0.285, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 1.1  , 2.   , 1.922, 0.561, 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.743, 1.95 , 2.   , 0.838, 0.   , 0.   , 0.   , 0.013,
       0.286, 0.168, 0.   , 0.   , 0.   , 0.   , 0.   , 0.203, 1.909,
       2.   , 1.3  , 0.039, 0.   , 0.   , 0.45 , 1.485, 1.996, 1.867,
       1.092, 0.   ,

In [52]:
def mse(w, X, Y, alpha):
    """
    étant donné qu'on a ajouté le bruit dans X à
    la première colonne de chaque exemple,
    ne pas oublier d'inclure le w_0 pour le bruit dans w.
    (dans le np.dot on calculera forcément 1*w_0 pour chaque exemple)
    """
    return np.mean((Y - np.dot(X, w.reshape(-1,1)))**2)

In [53]:
def mse_g(w, X, Y, alpha):
    """ retourne le gradient moyen de l'erreur au moindres carres """
    return np.hstack(
        np.dot(2*X.T, np.dot(X, w.reshape(-1,1)) - Y)
    )

In [54]:
def ridge_g(w, X, Y, alpha):
    """ retourne le gradient de ridge """
    return np.hstack(
        np.dot(2*X.T, np.dot(X, w.reshape(-1,1)) - Y) - alpha*2*w.reshape(-1,1)
    )

In [55]:
def lasso_g(w, X, Y, alpha):
    """ retourne le gradient de lasso """
    return np.hstack(
        np.dot(2*X.T, np.dot(X, w.reshape(-1,1)) - Y) - alpha*w.reshape(-1,1)/np.sum(np.abs(w))
    )

In [58]:
w = np.zeros(Xtrain.shape[1])
mse(w, Xtrain, Ytrain, None)

24.213962419421204

In [59]:
mse_g(w, Xtrain, Ytrain, None)

array([-8.74920e+04, -7.29100e+04, -5.83280e+04, ..., -2.32824e+02,
        0.00000e+00, -7.76080e+01])

In [60]:
ridge_g(w, Xtrain, Ytrain, 1)

array([-8.74920e+04, -7.29100e+04, -5.83280e+04, ..., -2.32824e+02,
        0.00000e+00, -7.76080e+01])

In [61]:
lasso_g(w, Xtrain, Ytrain, 1)

<ipython-input-55-9d160f4d7f0b>:4: RuntimeWarning: invalid value encountered in true_divide
  np.dot(2*X.T, np.dot(X, w.reshape(-1,1)) - Y) - alpha*w.reshape(-1,1)/np.sum(np.abs(w))


array([nan, nan, nan, ..., nan, nan, nan])

In [81]:
class Classifier(object):
    def __init__(self, loss, loss_g, alpha=0, max_iter=1000, eps=0.01):
        """ :loss: fonction de cout
            :loss_g: gradient de la fonction de cout
            :max_iter: nombre d'iterations
            :eps: pas de gradient
        """
        self.max_iter, self.eps = max_iter,eps
        self.loss, self.loss_g = loss, loss_g
        self.alpha = alpha
        self.w = None
        
    def fit(self, X, Y):
        Y = Y.reshape(-1,1)
        N = len(Y)
        X = X.reshape(N,-1)
        D = X.shape[1]
        self.w = np.zeros(D)
        for i in range(self.max_iter):
            self.w = self.w - self.eps * self.loss_g(self.w, X, Y, self.alpha)
        print(self.w)
        
    def predict(self, X):
        return np.hstack(np.sign(np.dot(X, self.w.reshape(-1,1))))
        
    def score(self):
        raise NotImplementedError

In [82]:
X = Xtrain
Y = Ytrain
d69X,d69Y = X[(Y==6)|(Y==9)],Y[(Y==6)|(Y==9)]
d69Y = np.where(d69Y==6,-1,1)

In [95]:
clf = Classifier(mse, mse_g, alpha=1, max_iter=10, eps=0.001)

In [96]:
clf.fit(d69X, d69Y)

[1.54399768e+21 8.19902688e+16 1.17183015e+18 3.45163535e+18
 1.48999158e+19 5.19200800e+19 1.92242416e+20 5.49469116e+20
 1.19977270e+21 1.87198003e+21 1.91503651e+21 1.22382725e+21
 5.11088330e+20 1.61614225e+20 5.34743977e+19 1.35367488e+19
 6.04235074e+17 1.16826615e+18 4.09406948e+18 2.38804668e+19
 9.08884109e+19 3.17542562e+20 7.92913319e+20 1.45370147e+21
 2.08345044e+21 2.37753431e+21 2.11016865e+21 1.63319303e+21
 1.03714789e+21 4.59094674e+20 1.60780876e+20 5.65838885e+19
 8.11031995e+18 2.88241845e+18 1.98740459e+19 7.99301914e+19
 2.84159650e+20 7.64785618e+20 1.34370232e+21 1.70306609e+21
 1.89840167e+21 1.58150905e+21 1.11595498e+21 1.08234969e+21
 1.00773223e+21 6.24733841e+20 2.59333604e+20 8.45959427e+19
 1.41170815e+19 6.97045373e+18 4.84108962e+19 1.81732931e+20
 5.55204976e+20 1.16770297e+21 1.50174895e+21 1.60375457e+21
 1.45943375e+21 9.36972280e+20 6.49081469e+20 8.59998120e+20
 9.76351732e+20 6.68403427e+20 2.94482533e+20 9.21606561e+19
 1.59590630e+19 1.361435